In [1]:
%load_ext autoreload
%autoreload 2


import os
import datetime
from glob import glob
from tqdm import tqdm

import cv2
import numpy as np
import mlflow
import matplotlib.pyplot as plt
import albumentations as A

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers,backend as K
from tensorflow.keras.metrics import Recall,Precision
import  tensorflow_addons as tfa
import logging

import one_ring
from one_ring.config import get_config
from one_ring.data import get_data_loader,get_camvid_data_loader
from one_ring.transformers import Transformer
from one_ring.models import Unet,DeepLabV3Plus,AttUnet
from one_ring.losses import FocalTverskyLoss, DiceLoss,BASNetHybridLoss,JaccardLoss,LogCoshDiceLoss,ComboLoss,BoundaryDoULoss
from one_ring.train import Trainer
from one_ring.callbacks import get_callbacks
from one_ring.losses import FocalTverskyLoss, DiceLoss,LogCoshDiceLoss,binary_focal_loss,categorical_focal_loss,FocalLoss,sym_unified_focal_loss,SymmetricUnifiedFocalLoss
from one_ring.metrics import DiceScore,JaccardScore
from one_ring.callbacks import ORLearningRateCallback
from one_ring.scheduler import ORLearningRateScheduler
from one_ring.utils import generate_overlay_image,calculate_confusion_matrix_and_report,plot_history_dict

print('tensorflow version :',tf.__version__)

import warnings

warnings.filterwarnings("always")
logging.getLogger('tensorflow').setLevel(logging.ERROR)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  

2024-09-09 01:49:13.958852: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-09 01:49:13.958877: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-09 01:49:13.959732: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-09 01:49:13.964722: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-09 01:49:14.615475: W tensorflow/compiler/tf2

tensorflow version : 2.15.0


/home/tuvis/miniconda3/envs/tf-gpu214/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
config = get_config(config_filename="spinal_cord")
cfg = config["augmentation"]

In [3]:

config = get_config(config_filename="spinal_cord")
train_data_loader, val_data_loader = get_data_loader(config.data, train_data=True, val_data=True, test_data=False)


IM_SIZE = config.data["image_size"][0]
aug_config = {
    "aug_prob": 0.1,
    #"random_contrast_limit": 0.4,
    #"random_brightness_limit": 0.3,
    "rotate_limit": 10,
}
tracing_object = {"mlflow":{"augmentation":aug_config}}

train_transforms = A.Compose(
    [
        A.Resize(IM_SIZE, IM_SIZE),
        #A.GaussNoise(var_limit=(10.0, 50.0), p=aug_config["aug_prob"]),
        #A.CLAHE(p=aug_config["aug_prob"]),
       # A.RandomBrightnessContrast(p=aug_config["aug_prob"], brightness_limit=aug_config["random_brightness_limit"], contrast_limit=aug_config["random_contrast_limit"]),
       # A.RandomGamma(p=aug_config["aug_prob"]),
        A.HorizontalFlip(p=aug_config["aug_prob"]),
        A.VerticalFlip(p=aug_config["aug_prob"]),
        A.Rotate(limit=aug_config["rotate_limit"], p=aug_config["aug_prob"]),
        A.RandomSizedCrop(min_max_height=(180, 224), height=IM_SIZE, width=IM_SIZE, p=aug_config["aug_prob"]),
    ]
)
test_transforms = A.Compose(
    [
        A.Resize(IM_SIZE, IM_SIZE),
        # A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]
)

tr_transforms_object = Transformer(config.augmentation, "train", train_transforms)#.from_dict()
ts_transforms_object = Transformer(config.augmentation, "test", test_transforms)#.from_dict()
train_dataset = train_data_loader.load_data(transform_func=tr_transforms_object)
val_dataset = val_data_loader.load_data(transform_func=ts_transforms_object, shuffle=False)

#callbacks = get_callbacks(config.callbacks)
callbacks = {}

steps_per_epoch = len(train_dataset)
lr_schedule = ORLearningRateScheduler(
    strategy=config.train.lr_scheduler["name"],
    total_epochs=config.train.epochs,
    steps_per_epoch=steps_per_epoch,
    **config.train.lr_scheduler["params"]
).get()
callbacks["lr_sch"] = ORLearningRateCallback(lr_schedule)

save_object = {
    "transformer":{
        "train":tr_transforms_object.to_dict(),
        "val": ts_transforms_object.to_dict()
    }
}

log_dir = f"board_logs/{config.train.experiment_name}/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
callbacks["tensorboard"] = tf.keras.callbacks.TensorBoard(log_dir=log_dir,histogram_freq=1, write_images=True, write_graph=True, update_freq='epoch')#, profile_batch='500,520')


2024-09-09 01:49:17.442545: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-09-09 01:49:17.468904: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-09-09 01:49:17.469051: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [4]:
loss_name = "boundary_dou_loss"
params = {
    "gamma": 4/3,
    "alpha": 0.4,
    "loss_weight": 0.4
}

tf.keras.backend.clear_session()

loss_dict = {
    "focal_loss": FocalLoss,
    "dice_loss": DiceLoss,
    "jaccard_loss": JaccardLoss,
    "log_cosh_dice_loss": LogCoshDiceLoss,
    "focal_tversky_loss": FocalTverskyLoss,
    "symmetric_unified_focal_loss": SymmetricUnifiedFocalLoss,
    "boundary_dou_loss": BoundaryDoULoss
}

metrics_dict= {
    "recal":Recall,
    "precision":Precision,
    "jaccard_score":JaccardScore
}

loss = loss_dict[loss_name](**params)
metrics = [m() for m in metrics_dict.values()]
callbacks = list(callbacks.values())

In [5]:
#model = AttUnet(**config.model).build_model()
model = DeepLabV3Plus(**config.model).build_model()
#model = Unet(**config.model).build_model()

#model.summary()
#keras.utils.plot_model(model, show_shapes=True, to_file=f'{model.name}_model.png')
trainer = Trainer(config, model, train_dataset, val_dataset, callbacks=liscallbacks, metrics=metrics,loss=loss,save_object=save_object)

8



/home/tuvis/tff/tf_seg/one_ring/logger.py:37: ResourceWarning: unclosed file <_io.TextIOWrapper name='/home/tuvis/tff/tf_seg/logs/trainer_log.log' mode='a' encoding='UTF-8'>
  logger.handlers = []


In [6]:
# #trainer.compile()
# print(trainer.loss)
# print(trainer.optimizer)
# print(trainer.callbacks)
# print(trainer.metrics)
history = trainer.fit()

Epoch 1:   0%|          | 0/49 [00:00<?, ?it/s]2024-09-09 01:49:33.015922: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inDeepLabV3Plus/EfficientNetB0_backbone/block2b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2024-09-09 01:49:34.355633: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2024-09-09 01:49:34.416185: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-09-09 01:49:35.449461: I external/local_xla/xla/service/service.cc:168] XLA service 0x7823b45dcea0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-09-09 01:49:35.449518: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 2070, Compute Capability 7.5
2024-09-09 01:49:35.455341: I tensorflow/co

In [7]:
trainer.finalize_training()

2024-09-09 01:50:07,711 - one_ring - INFO - Restored best model weights.
2024-09-09 01:50:20,883 - one_ring - INFO - Saved TensorFlow model to models/20240909014923/saved_model
INFO:one_ring:Saved TensorFlow model to models/20240909014923/saved_model
2024-09-09 01:50:20,891 - one_ring - INFO - Saved metadata to models/20240909014923/meta_data
INFO:one_ring:Saved metadata to models/20240909014923/meta_data
2024-09-09 01:50:21.939926: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-09-09 01:50:21.940030: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 1
2024-09-09 01:50:21.940089: I tensorflow/core/grappler/clusters/single_machine.cc:361] Starting new session
2024-0

In [ ]:
#trainer.load(trainer.save_path,setup_components=True)

In [16]:
model = trainer.model

In [13]:
def calculate_binary_differece(target, pred):
    # Calculate differences
    tp = (pred == 1) & (target == 1)  # True Positives
    fp = (pred == 1) & (target == 0)  # False Positives
    fn = (pred == 0) & (target == 1)  # False Negatives

    # Create an RGB image where each difference is colored differently
    # Initialize with zeros (black) for the background
    diff_image = np.zeros(target.shape + (3,), dtype=np.uint8)

    # Assign colors (R, G, B)
    # True Positives in green
    diff_image[tp] = [0, 255, 0]
    # False Positives in red
    diff_image[fp] = [255, 0, 0]
    # False Negatives in blue
    diff_image[fn] = [0, 0, 255]

    diff_image_corrected = np.squeeze(diff_image, axis=2)

    return diff_image_corrected

In [ ]:
threshold = 0.6
for i in val_dataset:
    pred_logits = model.predict(i[0])

    for n in range(len(i[0])): 
        image = i[0][n].numpy().astype(np.uint8)
        pred_logit = pred_logits[n]
     #   print(pred_logit.shape)
        pred_value = np.where(pred_logit>threshold,1,0).reshape(224,224,1)
        pred_mask = (pred_value*255).astype(np.uint8)

    #    print(pred_mask.shape, image.shape)
        overlay = generate_overlay_image(pred_mask, image, alpha=0.3)
        target = i[1][n].numpy()
        np.unique(target,return_counts=True)    

        plt.figure(figsize=(20,10))
        plt.subplot(1,3,1)
        plt.title('target')
        plt.imshow(target)
        plt.grid(True)
        plt.subplot(1,3,2)
        plt.title('pred')
        plt.imshow(pred_value)
        plt.grid(True)
        plt.subplot(1,3,3)
        plt.imshow(overlay)
        plt.grid(True)
        plt.show()

        #print(pred_value.shape, target.shape)

        cm,cr = calculate_confusion_matrix_and_report(pred_value, target)
        print(cm)
        print(cr)


In [ ]:
accuracy = tf.keras.metrics

In [ ]:
target = i[1][n]
pred_logit.shape,target.shape

a = accuracy(tf.squeeze(target),tf.squeeze(pred_logit))

In [ ]:
loss = DiceLoss()
from one_ring.losses import dice_coef

In [ ]:
pred_value = (pred_logit>0.5).astype(np.float32)


In [ ]:
dice_coef(target,pred_logit)

In [ ]:
dice_coef(target,pred_value)

In [ ]:
# remove dimension
tf.squeeze(target).shape

### Inferencer

In [ ]:
%load_ext autoreload
%autoreload 2

import os
import logging
import hashlib
from functools import lru_cache
from typing import Dict, Union, Optional, List, Tuple, Any

import numpy as np
from omegaconf import DictConfig, ListConfig
import matplotlib.pyplot as plt 
import cv2

from one_ring.deploy.inferencer import Inferencer

# from one_ring.utils import TensorLike
# from one_ring.config import MODEL_TYPE_LIB
# from one_ring.transformers import normalize
# from one_ring.deploy import model_wrapper_lib,preprocessor_lib,postprocessor_lib
# from one_ring.deploy.processing import VanillaPostprocessor

# import json
# import yaml

In [ ]:
config = {
    #"image_size": [224, 224, 3],
    "model_type": "onnx",
    "model_path": "models/20240909002117",
    "preprocessor_type": "albumentations",
    "postprocessor_type": "vanilla",
    "device": "cpu",
    "threshold":0.6
}


inferencer = Inferencer(config, cache_size=256, log_level="INFO")
#inferencer.load_processor("preprocessor")

# inferencer.preprocessor = preprocessor
# inferencer.postprocessor = postprocessor

In [ ]:
image_path = "examples/test_images/road_2.jpg"
image = plt.imread(image_path)
plt.imshow(image)
plt.show()
res = inferencer(image)

In [38]:
s = inferencer.preprocessor.transform(image=image)["image"]

In [ ]:
plt.imshow(res[1][0])

#### Sequence Exp